Note: This jupyter notebook was adapted from my initial work done on a Google Collab space, therefore some implementations reflect that.

### Installations

The following are the necessary installations required to run this notebook.

Make sure you have the `requirements.txt` file!!!

Change the path accordingly to where your file is stored

```python
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
```